Create Widgets for Silver Table

In [0]:
dbutils.widgets.text("catalog", "workspace")
dbutils.widgets.text("schema", "silver")
dbutils.widgets.text("table", "match_first_point_winner")

Silver to Gold Transformation

In [0]:
#Compare point winner to match winner
from pyspark.sql.functions import when, count, lit, col, round

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
table = dbutils.widgets.get("table")


silver_df = spark.table(f"{catalog}.{schema}.{table}")

rally_df = silver_df.filter(col("RallyCount") > 3)


analysis_df = rally_df.withColumn("first_point_wins_match",
                                  when(col("first_point_winner") == col("match_winner"), True)
                                  .otherwise(False)
                                  )

#display(analysis_df.limit(10))

total = analysis_df.count()

win_rate_df = (
    analysis_df
    .groupBy("first_point_wins_match")
    .agg(
        (count("match_id") / lit(total)).alias("win_percentage")
    )
)

win_rate_df = win_rate_df.withColumn(
    "win_percentage",
    round(col("win_percentage") * 100, 2)
)

# display(win_rate_df)

Create Gold Table

In [0]:
#Create schema and table
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.gold")

(
    win_rate_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{catalog}.gold.match_first_point_winner_rate")
)
